train-tfidf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=33b0611cc3062fc4ba5f8be2ac5f6f038e187386975e144d277c2ac05ff996f8
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.6 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-4-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-5-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd


In [ ]:
bodo_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/train_BO_AH_HASOC2023.csv')

In [ ]:
bodo_train['task_1'].value_counts()

HOF    998
NOT    681
Name: task_1, dtype: int64

In [ ]:
bodo_train['clean_text'] = bodo_train['text'].apply(lambda x:emo(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:remove_urls(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
#To split train and test
X_train, X_test, y_train, y_test = train_test_split(bodo_train.clean_text, bodo_train.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((1259,), (420,))

In [ ]:
import regex
def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) #extract words of at least 2 letters
#     for w in words:
    for w in words:
      yield w

In [ ]:
#feature extraction
# word level tf-idf
tfidf_vect1 = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vect1.fit(X_train)
xtrain_tfidf1 =  tfidf_vect1.transform(X_train)
xtest_tfidf1=  tfidf_vect1.transform(X_test)

In [ ]:
xtrain_tfidf1 .shape,xtest_tfidf1.shape

((1259, 3844), (420, 3844))

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(xtrain_tfidf1, y_train)

y_pred = svm.predict(xtest_tfidf1)

accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))

Test Accuracy: 76.9048

               precision    recall  f1-score   support

         HOF       0.73      0.94      0.82       233
         NOT       0.88      0.56      0.68       187

    accuracy                           0.77       420
   macro avg       0.80      0.75      0.75       420
weighted avg       0.79      0.77      0.76       420



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(xtrain_tfidf1, y_train)
y_pred = eclf.predict(xtest_tfidf1)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))